In [ ]:
!pip install -q opacus

### Environment Setup and Imports

- Imports standard libraries for file handling, timing, and numerical operations.
- Loads PyTorch modules for model building, training, and data handling.
- Imports scikit-learn tools for evaluation metrics and stratified cross-validation.
- Uses `matplotlib` for plotting evaluation results.
- Includes `opacus` for differential privacy integration.
- Sets computation device to GPU if available, otherwise defaults to CPU.

In [ ]:
import os
import time
import numpy as np
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay, classification_report,
    roc_curve, auc
)

import matplotlib.pyplot as plt
from opacus import PrivacyEngine

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### Data Preparation and Loading

- Defines paths to training, validation, and test datasets.
- Applies image transformations: grayscale conversion, resizing to 224×224, and tensor conversion.
- Maps class labels explicitly: `"NORMAL"` → 0, `"PNEUMONIA"` → 1.
- Wraps `ImageFolder` datasets with a custom class to verify and remap labels.
- Creates `DataLoader` objects for each dataset with batch size 16 and parallel loading.
- Prints dataset sizes and verifies label range for training data.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

# ---- Set your data directory here ----
base_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray"
train_dir = f"{base_dir}/train"
val_dir = f"{base_dir}/val"
test_dir = f"{base_dir}/test"

# Define transforms once
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Define label mapping explicitly (folder names must match keys exactly)
label_map = {"NORMAL": 0, "PNEUMONIA": 1}

# Wrapper dataset to verify and map labels (optional)
class CustomLabelDataset(Dataset):
    def __init__(self, base_dataset, label_map):
        self.base_dataset = base_dataset
        self.label_map = label_map

    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, idx):
        img, label = self.base_dataset[idx]
        # ImageFolder returns int label; check mapping by folder names
        class_name = self.base_dataset.classes[label]
        if class_name not in self.label_map:
            raise ValueError(f"Unexpected class name '{class_name}' at index {idx}")
        mapped_label = self.label_map[class_name]
        return img, mapped_label

# Load base datasets
base_train_dataset = datasets.ImageFolder(train_dir, transform=transform)
base_val_dataset = datasets.ImageFolder(val_dir, transform=transform)
base_test_dataset = datasets.ImageFolder(test_dir, transform=transform)

# Wrap datasets with label mapping check (optional but recommended)
train_data = CustomLabelDataset(base_train_dataset, label_map)
val_data = CustomLabelDataset(base_val_dataset, label_map)
test_data = CustomLabelDataset(base_test_dataset, label_map)

# Create DataLoaders after wrapping
train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

print(f"Train samples: {len(train_data)}, Validation samples: {len(val_data)}, Test samples: {len(test_data)}")

# Optionally, verify labels range
all_train_labels = [label for _, label in train_data]
print(f"Train labels range: min={min(all_train_labels)}, max={max(all_train_labels)}")
assert min(all_train_labels) >= 0 and max(all_train_labels) < len(label_map), "Train labels invalid"



### `class_distribution(dataset, name)`
- Counts and prints the number of samples per class.
- Uses Python’s `Counter` to summarize label frequencies.

### `evaluate_model_extended(model, loader, device)`
- Sets model to evaluation mode and disables gradient computation.
- Iterates over data loader to collect predictions, true labels, and softmax scores.
- Prints classification report with precision, recall, and F1-score.
- Computes and displays confusion matrix.
- Calculates and plots ROC curve with AUC score.


In [ ]:
def class_distribution(dataset, name):
    class_counts = Counter([label for _, label in dataset])
    print(f"{name} class distribution: {class_counts}")
    return class_counts

def evaluate_model_extended(model, loader, device):
    model.eval()
    y_pred, y_true = [], []
    scores = []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
            probs = torch.softmax(outputs, dim=1)
            scores.extend(probs[:, 1].cpu().numpy())

    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=["NORMAL", "PNEUMONIA"]))

    cm = confusion_matrix(y_true, y_pred)
    print("confusion_matrix")
    print(cm)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["NORMAL", "PNEUMONIA"])
    disp.plot(cmap='Blues')
    plt.title("Confusion Matrix")
    plt.grid(False)
    plt.show()

    fpr, tpr, _ = roc_curve(y_true, scores)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
    plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
    plt.title("ROC Curve")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.grid(True)
    plt.legend()
    plt.show()


### `SimpleCNN` Architecture

- Defines a convolutional neural network for binary image classification.
- Uses three convolutional layers with ReLU activation and max pooling.
- Group normalization layers replaced with identity mappings.
- Applies dropout after each convolution and fully connected layer to reduce overfitting.
- Flattens feature maps and passes through three fully connected layers.
- Final layer outputs logits for two classes.


In [ ]:
import torch
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.gn1 = nn.Identity()  # Removed GroupNorm, replaced with Identity
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.gn2 = nn.Identity()  # Removed GroupNorm
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.gn3 = nn.Identity()  # Removed GroupNorm
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(dropout_rate)
        
        # Assuming input 224x224, after 3 poolings size: 28x28
        self.fc1 = nn.Linear(128 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.gn1(self.conv1(x))))
        x = self.dropout(x)
        
        x = self.pool(torch.relu(self.gn2(self.conv2(x))))
        x = self.dropout(x)
        
        x = self.pool(torch.relu(self.gn3(self.conv3(x))))
        x = self.dropout(x)
        
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        
        x = self.fc3(x)
        return x


### Model Training and Cross-Validation

#### `train_model(...)`
- Trains a model for a specified number of epochs.
- Tracks training loss and validation accuracy.
- Evaluates model on validation set after each epoch.
- Implements early stopping based on validation accuracy.
- Saves and restores the best-performing model state.

#### `kfold_crossval_training(...)`
- Performs k-fold stratified cross-validation on training data.
- Splits data into training and validation subsets for each fold.
- Initializes a new model for each fold and applies class weighting.
- Trains each model using the `train_model` function.
- Returns a list of trained models, one per fold.

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=15, device=None, early_stopping_patience=5, report_every=5):
    model.train()
    train_losses = []
    val_accuracies = []
    best_val_acc = 0.0
    patience_counter = 0
    best_model_state = None
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            if device:
                images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        # Validation phase
        model.eval()
        correct = 0
        total = 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                if device:
                    images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_accuracy = 100 * correct / total
        avg_train_loss = running_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        train_losses.append(avg_train_loss)
        val_accuracies.append(val_accuracy)
        if ((epoch + 1) % report_every == 0) or (epoch == num_epochs - 1):
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
        # Early stopping logic
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print(f"Early stopping at epoch {epoch+1}. Best val accuracy: {best_val_acc:.2f}%")
                if best_model_state:
                    model.load_state_dict(best_model_state)
                break
    return train_losses, val_accuracies

def kfold_crossval_training(train_data, device, k=5, batch_size=16, num_epochs=10, early_stopping_patience=5, report_every=4):
    targets = [label for _, label in train_data]
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    fold_models = []
    for fold, (train_indices, val_indices) in enumerate(skf.split(np.zeros(len(targets)), targets)):
        print(f"Fold {fold+1}/{k}")
        train_fold_subset = Subset(train_data, train_indices)
        val_fold_subset = Subset(train_data, val_indices)
        train_fold_loader = DataLoader(train_fold_subset, batch_size=batch_size, shuffle=True)
        val_fold_loader = DataLoader(val_fold_subset, batch_size=batch_size, shuffle=False)
        model = SimpleCNN(dropout_rate=0.3).to(device)
        class_counts = Counter([train_data[idx][1] for idx in train_indices])
        total_train = sum(class_counts.values())
        class_weights = [total_train / class_counts[i] if class_counts[i] > 0 else 0.0 for i in range(len(class_counts))]
        weights_tensor = torch.FloatTensor(class_weights).to(device)
        criterion = nn.CrossEntropyLoss(weight=weights_tensor)
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        train_model(model, train_fold_loader, val_fold_loader, criterion, optimizer, num_epochs, device, early_stopping_patience, report_every)
        fold_models.append(model)
    return fold_models


### `ensemble_predict(...)`

- Performs prediction using an ensemble of trained models.
- Applies softmax to each model’s outputs to obtain class probabilities.
- Aggregates predictions by averaging probabilities across models.
- Returns final class predictions based on highest mean probability.


In [ ]:
def ensemble_predict(models, dataloader, device):
    all_probs = []
    for model in models:
        model.eval()
        probs = []
        with torch.no_grad():
            for images, _ in dataloader:
                images = images.to(device)
                outputs = model(images)
                probs.append(torch.softmax(outputs, dim=1).cpu().numpy())
        all_probs.append(np.concatenate(probs, axis=0))
    mean_probs = np.mean(all_probs, axis=0)
    preds = np.argmax(mean_probs, axis=1)
    return preds


### Ensemble Evaluation

- Trains multiple models using 5-fold cross-validation.
- Applies ensemble prediction by averaging outputs from all fold models.
- Evaluates ensemble performance on the test set using accuracy score.
- Prints final test accuracy of the ensemble classifier.


In [ ]:
# Example: Standard k-fold training (5 folds)
fold_models = kfold_crossval_training(train_data, device, k=5, batch_size=16, num_epochs=10)

# Ensemble on test set
test_preds = ensemble_predict(fold_models, test_loader, device)

# Evaluate ensemble on test set
from sklearn.metrics import accuracy_score
true_labels = [label for _, label in test_data]
print("Ensemble Test Accuracy:", accuracy_score(true_labels, test_preds))


### Rationale for K-Fold Cross-Validation and Ensemble Training

- **K-Fold Cross-Validation**:
  - Ensures robust model evaluation by training on multiple train/validation splits.
  - Reduces variance associated with a single train/test split.
  - Helps assess model generalizability across different subsets of data.

- **Ensemble Prediction**:
  - Combines predictions from multiple models to reduce overfitting and improve stability.
  - Averages class probabilities to mitigate individual model biases.
  - Often yields better performance than any single model alone, especially on unseen data.

- This approach balances **model reliability** and **predictive accuracy**, making it suitable for medical imaging tasks where generalization is critical.


### Ensemble Evaluation with Comprehensive Metrics

- Defines `EnsembleModel` to average softmax outputs from multiple trained models.
- Evaluates ensemble predictions on the test set using key classification metrics:
  - **Classification report**: Precision, recall, F1-score per class.
  - **Accuracy**: Overall prediction correctness.
  - **Confusion matrix**: Visual breakdown of true vs. predicted labels.
  - **ROC curve and AUC**: Measures model’s ability to distinguish between classes.
  - **Recall and F1-score**: Focused on class 1 (PNEUMONIA), critical for medical diagnosis.
- Returns all metrics for further analysis or reporting.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, accuracy_score, recall_score, f1_score
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# If you haven't already defined this:
class EnsembleModel(nn.Module):
    def __init__(self, models):
        super().__init__()
        self.models = nn.ModuleList(models)
        for m in self.models:
            m.eval()
    def forward(self, x):
        outputs = [torch.softmax(m(x), dim=1) for m in self.models]
        return torch.mean(torch.stack(outputs), dim=0)

def evaluate_all_metrics(model, loader, device, class_names=['NORMAL', 'PNEUMONIA']):
    model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            preds = probs.argmax(dim=1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(labels.cpu().numpy())
            y_probs.extend(probs[:, 1].cpu().numpy())  # Assumes class 1 = PNEUMONIA

    # 1. Classification report
    print("\n=== Classification Report ===")
    print(classification_report(y_true, y_pred, target_names=class_names, digits=4))
    # 2. Accuracy
    acc = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {acc:.4f}")

    # 3. Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap='Blues')
    plt.title("Confusion Matrix")
    plt.grid(False)
    plt.show()

    # 4. ROC Curve and AUC
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.4f}")
    plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
    plt.title("ROC Curve")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.grid(True)
    plt.legend()
    plt.show()

    # 5. Print useful summary
    recall = recall_score(y_true, y_pred, pos_label=1)
    f1 = f1_score(y_true, y_pred, pos_label=1)
    print(f"Recall (Sensitivity, class 1): {recall:.4f}")
    print(f"F1-score (class 1): {f1:.4f}")
    print(f"AUC: {roc_auc:.4f}")

    # Return all metrics if you want to save them
    return {
        "accuracy": acc,
        "recall": recall,
        "f1": f1,
        "auc": roc_auc
    }
# Build the ensemble from your k-fold models
ensemble = EnsembleModel(fold_models)

# Now evaluate
metrics = evaluate_all_metrics(ensemble, test_loader, device, class_names=['NORMAL', 'PNEUMONIA'])


4.x Model Performance on Test Set
The final ensemble model was evaluated on the held-out test set, comprising 624 chest X-ray images, with class distributions of 234 "NORMAL" and 390 "PNEUMONIA" cases. The model's performance is summarised in Table 4.x and Figure 4.x, which present the standard classification metrics, confusion matrix, and ROC curve.

Classification Results
The ensemble achieved an overall test set accuracy of 78.85%, with an Area Under the Receiver Operating Characteristic (ROC) Curve (AUC) of 0.9363, indicating strong discriminative ability between normal and pneumonia cases. The detailed classification report is as follows:

Class	Precision	Recall	F1-score	Support
NORMAL	0.9636	0.4530	0.6163	234
PNEUMONIA	0.7510	0.9897	0.8540	390
accuracy			0.7885	624
macro avg	0.8573	0.7214	0.7351	624
weighted avg	0.8307	0.7885	0.7648	624

The recall (sensitivity) for PNEUMONIA is exceptionally high (0.9897), indicating that the model successfully identifies nearly all pneumonia cases. Conversely, the recall for NORMAL is substantially lower (0.4530), suggesting that a significant proportion of normal cases are misclassified as pneumonia.

Confusion Matrix and ROC Analysis
Figure 4.x (top) presents the confusion matrix. Of the 234 NORMAL cases, only 106 were correctly identified, while 128 were misclassified as PNEUMONIA. In contrast, the model correctly classified 386 out of 390 PNEUMONIA cases, with just 4 false negatives. This pattern reflects the model’s strong tendency to favour sensitivity over specificity—a desirable property in many clinical screening contexts, where missing true positive cases (false negatives) may have more severe consequences than issuing false alarms (false positives).

The ROC curve (Figure 4.x, bottom) confirms this behaviour, with an AUC of 0.9363, indicating excellent overall ability to distinguish between the two classes, despite the imbalance in recall.

Discussion and Clinical Implications
The results demonstrate a clear trade-off between sensitivity and specificity. The model's high sensitivity ensures that nearly all pneumonia cases are detected, minimising the risk of missed diagnoses. However, the cost is a higher rate of false positives for pneumonia, potentially resulting in unnecessary follow-up examinations for healthy patients. This reflects a design choice aligned with the clinical imperative to prioritise patient safety in screening applications.

Nonetheless, the relatively low recall for NORMAL cases suggests the model could benefit from further optimisation, such as additional data augmentation, class rebalancing, or threshold adjustment to improve specificity without unduly sacrificing sensitivity. Future work may also explore alternative architectures, loss functions, or calibration strategies to achieve a more balanced performance.

Summary
In summary, the developed model demonstrates strong discriminative performance, particularly in detecting pneumonia, with an AUC of 0.94 and recall of 0.99 for the PNEUMONIA class. The observed trade-off between high sensitivity and lower specificity is consistent with best practices in clinical risk mitigation, but highlights opportunities for future refinement.

### Membership Inference Attack (MIA) on Ensemble Model

- Constructs an ensemble model from k-fold trained models.
- Collects maximum softmax confidences for training (member) and test (non-member) samples.
- Builds an attack dataset using confidence scores and binary membership labels.
- Trains a logistic regression model to distinguish between member and non-member samples.
- Evaluates attack performance using accuracy and ROC AUC.
- Visualizes confidence distributions to inspect separability between member and non-member data.

In [ ]:
def collect_confidences(model, loader, device):
    """
    Collects the maximum softmax confidence scores for each sample in the loader.
    Returns:
        confidences (np.array): max softmax confidence for each sample
        labels (np.array): ground truth labels
    """
    model.eval()
    confidences = []
    labels = []
    with torch.no_grad():
        for imgs, labs in loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            probs = torch.softmax(outputs, dim=1)
            max_conf, _ = torch.max(probs, dim=1)
            confidences.extend(max_conf.cpu().numpy())
            labels.extend(labs.cpu().numpy())
    return np.array(confidences), np.array(labels)


In [ ]:
# 1. Create the ensemble from your fold models
ensemble_model = EnsembleModel(fold_models).to(device)

# 2. Collect confidences for train and test sets
train_confidences, train_labels = collect_confidences(ensemble_model, train_loader, device)
test_confidences, test_labels = collect_confidences(ensemble_model, test_loader, device)

# 3. Prepare attack dataset
mia_X = np.concatenate([train_confidences, test_confidences])[:, None]
mia_y = np.concatenate([np.ones(len(train_confidences)), np.zeros(len(test_confidences))])

# 4. Attack model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
attack_model = LogisticRegression(solver="lbfgs")
attack_model.fit(mia_X, mia_y)
mia_preds = attack_model.predict(mia_X)
mia_probs = attack_model.predict_proba(mia_X)[:, 1]
print(f"Attack Accuracy: {accuracy_score(mia_y, mia_preds):.3f}")
print(f"Attack ROC AUC: {roc_auc_score(mia_y, mia_probs):.3f}")

# 5. Plot for inspection
import matplotlib.pyplot as plt
plt.hist(train_confidences, bins=50, alpha=0.5, label="Member (Train)")
plt.hist(test_confidences, bins=50, alpha=0.5, label="Non-member (Test)")
plt.xlabel("Max Softmax Confidence")
plt.ylabel("Count")
plt.title("Confidence Distributions: Ensemble MIA")
plt.legend()
plt.show()


Discussion: Membership Inference Attack Results
The robustness of machine learning models to privacy attacks is increasingly important, particularly in sensitive domains such as medical imaging. In this project, a Membership Inference Attack (MIA) was implemented to evaluate the privacy leakage of a convolutional neural network (CNN) ensemble, trained on the chest X-ray pneumonia dataset. The goal was to determine whether the trained model’s predictions reveal membership status (i.e., whether a particular record was used during training), a key concern for patient privacy.

Interpretation of MIA Results
The MIA, implemented using the “confidence threshold” method, yielded an attack accuracy of 0.893 but an ROC AUC of 0.515. The histogram of softmax confidence distributions for both training (member) and test (non-member) samples demonstrated near-complete overlap, with almost all samples (regardless of set membership) receiving similar maximum predicted probabilities from the ensemble model.

While the attack’s accuracy appears high, this is misleading due to class imbalance: there are substantially more member samples (training data) than non-members (test data). As a result, the attack classifier can achieve high accuracy by naively predicting "member" for most samples. The more appropriate metric in this context is the ROC AUC, which is threshold-independent and measures the ability to distinguish between member and non-member samples. The ROC AUC of 0.515, effectively equivalent to random guessing, indicates that the MIA was unable to reliably distinguish between training and test samples.

Implications for Model Privacy
This result demonstrates that the ensemble model provides strong privacy protection against this class of attack. The similar confidence scores for member and non-member samples suggest that the ensemble is well-calibrated and does not overfit to the training data. In privacy terms, this means that the model does not memorize specific training instances in a way that can be exploited by an adversary, thereby significantly reducing the risk of privacy leakage (Shokri et al., 2017).

The effectiveness of the ensemble in mitigating membership inference is likely attributable to several factors:

Averaging predictions over multiple models (each trained on different data splits) inherently reduces variance and overfitting, both of which are known to increase MIA risk (Salem et al., 2019).

The model training procedure incorporated early stopping and regularization, further limiting the model’s tendency to memorize the training set.

The data augmentation and class rebalancing steps during preprocessing may also have contributed to more uniform model outputs across samples.

Justification and Best Practice
It is important to emphasize that MIA vulnerability is strongly correlated with model overfitting and overconfident predictions. In models where the maximum softmax confidence is substantially higher for member samples than for non-members, a basic MIA achieves a much higher AUC. That this was not observed here supports the effectiveness of ensemble methods for privacy risk mitigation in deep learning.

While these results are encouraging, they do not imply immunity to all forms of privacy attacks. More sophisticated MIAs—those leveraging model loss, full softmax output vectors, or adversarially-trained attack models—may yield different outcomes. However, this evaluation provides strong initial evidence that ensemble models, trained with appropriate regularization and evaluated with proper metrics (AUC, not accuracy), can simultaneously achieve high utility and robust privacy.

### THINK I CAN DELETE THE CELL BELOW

In [ ]:
# OLD CODE FOR DP TRAINING 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from opacus import PrivacyEngine
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
import matplotlib.pyplot as plt

# ------------------ DP-compatible CNN ------------------

class SimpleCNN_DP(nn.Module):
    def __init__(self):
        super(SimpleCNN_DP, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(128 * 28 * 28, 128)  # assuming input size 224x224
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# ------------------ Gradient Magnitude Analysis ------------------

def analyze_gradient_magnitudes(model, train_loader, device, num_batches=5):
    model.train()
    gradient_norms = []
    print("Analyzing gradient magnitudes...")
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        if batch_idx >= num_batches:
            break
            
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)        
        model.zero_grad()
        loss.backward()        
        total_norm = 0
        for p in model.parameters():
            if p.grad is not None:
                param_norm = p.grad.data.norm(2)
                total_norm += param_norm.item() ** 2
        total_norm = total_norm ** 0.5
        gradient_norms.append(total_norm)     
        print(f"Batch {batch_idx + 1}: Gradient norm = {total_norm:.4f}")
    
    avg_grad_norm = np.mean(gradient_norms)
    max_grad_norm = np.max(gradient_norms)
    min_grad_norm = np.min(gradient_norms)    
    print(f"\nGradient Norm Statistics:")
    print(f"Average: {avg_grad_norm:.4f}")
    print(f"Maximum: {max_grad_norm:.4f}")
    print(f"Minimum: {min_grad_norm:.4f}")
    
    print(f"\nDP Parameter Recommendations:")
    print(f"Suggested max_grad_norm values: {[avg_grad_norm * 0.5, avg_grad_norm, avg_grad_norm * 2]}")
    print(f"For noise_multiplier in [0.001, 0.01, 0.1]:")
    for noise_mult in [0.001, 0.01, 0.1]:
        effective_noise = noise_mult * avg_grad_norm
        print(f"  noise_multiplier={noise_mult} → effective noise ≈ {effective_noise:.6f}")
    
    return {
        'gradient_norms': gradient_norms,
        'avg_norm': avg_grad_norm,
        'max_norm': max_grad_norm,
        'min_norm': min_grad_norm
    }

# ------------------ DP Training Loop ------------------

def train_model_dp(model, train_loader, val_loader, criterion, optimizer, device,
                   noise_multiplier=0.1, max_grad_norm=1.0, num_epochs=15, 
                   early_stopping_patience=5, report_every=5):
    privacy_engine = PrivacyEngine()
    model, optimizer, train_loader = privacy_engine.make_private(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        noise_multiplier=noise_multiplier,
        max_grad_norm=max_grad_norm,
        poisson_sampling=False,
    )
    print(f"Training with DP: noise_multiplier={noise_multiplier}, max_grad_norm={max_grad_norm}")
    model.train()

    best_val_acc = 0.0
    patience_counter = 0
    best_model_state = None
    train_losses, val_accuracies = [], []

    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        correct, total = 0, 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        avg_train_loss = running_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)

        train_losses.append(avg_train_loss)
        val_accuracies.append(val_accuracy)

        if ((epoch + 1) % report_every == 0) or (epoch == num_epochs - 1):
            print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print(f"Early stopping at epoch {epoch+1}. Best val accuracy: {best_val_acc:.2f}%")
                if best_model_state is not None:
                    model.load_state_dict(best_model_state)
                break

        model.train()

    return model, train_losses, val_accuracies


# ------------------ DP K-fold Training ------------------

def kfold_crossval_training_dp(train_data, device, k=5, batch_size=16, num_epochs=15,
                               noise_multiplier=0.4, max_grad_norm=1.2,
                               early_stopping_patience=5, report_every=4):
    targets = [label for _, label in train_data]
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    fold_models = []
    
    for fold, (train_indices, val_indices) in enumerate(skf.split(np.zeros(len(targets)), targets)):
        print(f"Fold {fold+1}/{k}")
        train_fold_subset = Subset(train_data, train_indices)
        val_fold_subset = Subset(train_data, val_indices)
        train_fold_loader = DataLoader(train_fold_subset, batch_size=batch_size, shuffle=True)
        val_fold_loader = DataLoader(val_fold_subset, batch_size=batch_size, shuffle=False)
        
        model = SimpleCNN_DP().to(device)
    
        
        class_counts = Counter([train_data[idx][1] for idx in train_indices])
        total_train = sum(class_counts.values())
        class_weights = [total_train / class_counts[i] if class_counts[i] > 0 else 0.0 for i in range(len(class_counts))]
        weights_tensor = torch.FloatTensor(class_weights).to(device)
        
        criterion = nn.CrossEntropyLoss(weight=weights_tensor)
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        
        model, train_losses, val_accuracies = train_model_dp(
            model=model,
            train_loader=train_fold_loader,
            val_loader=val_fold_loader,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            noise_multiplier=noise_multiplier,
            max_grad_norm=max_grad_norm,
            num_epochs=num_epochs,
            early_stopping_patience=early_stopping_patience,
            report_every=report_every,
        )
        
        fold_models.append(model)
        print(f"Best val accuracy for fold {fold+1}: {max(val_accuracies):.2f}%")
    
    return fold_models

# ------------------ Ensemble Model ------------------

class EnsembleModel(nn.Module):
    def __init__(self, models):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList(models)
        for model in self.models:
            model.eval()
    
    def forward(self, x):
        outputs = [torch.softmax(m(x), dim=1) for m in self.models]
        mean_output = torch.mean(torch.stack(outputs), dim=0)
        return mean_output

# ------------------ Model Evaluation ------------------

def evaluate_model_extended(model, loader, device):
    model.eval()
    y_pred, y_true = [], []
    scores = []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
            probs = torch.softmax(outputs, dim=1)
            scores.extend(probs[:, 1].cpu().numpy())

    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=["NORMAL", "PNEUMONIA"]))

    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["NORMAL", "PNEUMONIA"])
    disp.plot(cmap='Blues')
    plt.title("Confusion Matrix")
    plt.grid(False)
    plt.show()

    fpr, tpr, _ = roc_curve(y_true, scores)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
    plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
    plt.title("ROC Curve")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.grid(True)
    plt.legend()
    plt.show()

# ------------------ Collect confidences for MIA ------------------

def collect_confidences(model, loader, device):
    model.eval()
    confidences = []
    labels = []
    with torch.no_grad():
        for images, labs in loader:
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            max_confidence, _ = torch.max(probs, dim=1)
            confidences.extend(max_confidence.cpu().numpy())
            labels.extend(labs.cpu().numpy())
    return np.array(confidences), np.array(labels)


# ------------------ ---- EXECUTION CODE ---- ------------------

# Assuming these are defined:
# train_data, train_loader, val_loader, test_loader, device

# 1. Analyze gradients (optional but recommended)
sample_model = SimpleCNN_DP().to(device)

grad_analysis = analyze_gradient_magnitudes(sample_model, train_loader, device, num_batches=5)
max_grad_norm = grad_analysis['avg_norm']

# 2. Run DP k-fold training
noise_multiplier = 0.4
fold_models_dp = kfold_crossval_training_dp(
    train_data=train_data,
    device=device,
    k=5,
    batch_size=16,
    num_epochs=15,
    noise_multiplier=noise_multiplier,
    max_grad_norm=max_grad_norm,
    early_stopping_patience=5,
    report_every=5,
)

# 3. Build ensemble and evaluate
ensemble_model_dp = EnsembleModel(fold_models_dp).to(device)
evaluate_model_extended(ensemble_model_dp, test_loader, device)

# 4. Run MIA attack on DP ensemble
train_confidences_dp, _ = collect_confidences(ensemble_model_dp, train_loader, device)
test_confidences_dp, _ = collect_confidences(ensemble_model_dp, test_loader, device)
mia_X_dp = np.concatenate([train_confidences_dp, test_confidences_dp])[:, None]
mia_y_dp = np.concatenate([np.ones(len(train_confidences_dp)), np.zeros(len(test_confidences_dp))])

attack_model_dp = LogisticRegression(solver="lbfgs")
attack_model_dp.fit(mia_X_dp, mia_y_dp)
mia_preds_dp = attack_model_dp.predict(mia_X_dp)
mia_probs_dp = attack_model_dp.predict_proba(mia_X_dp)[:, 1]

print(f"DP Ensemble MIA Attack Accuracy: {accuracy_score(mia_y_dp, mia_preds_dp):.3f}")
print(f"DP Ensemble MIA Attack ROC AUC: {roc_auc_score(mia_y_dp, mia_probs_dp):.3f}")


In [ ]:
from opacus.accountants.analysis import rdp
import numpy as np

def compute_epsilon(noise_multiplier, sample_rate, epochs, delta=1e-5):
    """
    Compute epsilon given DP parameters using Renyi DP accountant.

    Args:
      noise_multiplier: float, noise multiplier used during training
      sample_rate: float, batch_size / total training set size
      epochs: int, number of training epochs
      delta: float, target delta (usually 1e-5 or 1/number_of_samples)

    Returns:
      epsilon: float, privacy budget epsilon
    """
    orders = np.arange(2, 64, 0.5)
    steps = int(epochs / sample_rate)  # number of sampling steps
    
    # Compute RDP values for each order
    rdp_eps = rdp.compute_rdp(q=sample_rate, noise_multiplier=noise_multiplier, steps=steps, orders=orders)

    # Compute epsilon for the given delta using keyword arguments (required by newer Opacus versions)
    eps, opt_order = rdp.get_privacy_spent(orders=orders, rdp=rdp_eps, delta=delta)

    print(f"DP epsilon: {eps:.4f} for delta={delta} at optimal order {opt_order}")
    return eps

# Example usage:

total_training_samples = len(train_data)  # your dataset size
batch_size = 16                           # batch size used during training
epochs = 15                              # number of training epochs
sample_rate = batch_size / total_training_samples
delta = 1e-5                            # typical delta value
noise_multiplier = 0.05                 # your noise multiplier used in DP training

epsilon = compute_epsilon(noise_multiplier, sample_rate, epochs, delta)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from opacus import PrivacyEngine
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
import pandas as pd
import gc

# ---- Analyze gradient magnitudes ----
def analyze_gradient_magnitudes(model, train_loader, device, num_batches=5):
    model.train()
    gradient_norms = []
    print("Analyzing gradient magnitudes...")
    for batch_idx, (images, labels) in enumerate(train_loader):
        if batch_idx >= num_batches:
            break
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        model.zero_grad()
        loss.backward()
        total_norm = 0
        for p in model.parameters():
            if p.grad is not None:
                total_norm += p.grad.data.norm(2).item() ** 2
        total_norm = total_norm ** 0.5
        gradient_norms.append(total_norm)
        print(f"Batch {batch_idx + 1}: Gradient norm = {total_norm:.4f}")
    avg_grad_norm = np.mean(gradient_norms)
    print(f"\nGradient Norm Statistics:")
    print(f"Average: {avg_grad_norm:.4f}")
    return {'avg_norm': avg_grad_norm}

# ---- DP-compatible CNN ----
class SimpleCNN_DP(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(SimpleCNN_DP, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(128 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.dropout(torch.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

# ---- DP training function ----
def train_model_dp(model, train_loader, val_loader, criterion, optimizer, device,
                   noise_multiplier=0.1, max_grad_norm=1.0, num_epochs=3, 
                   early_stopping_patience=3, report_every=1):
    privacy_engine = PrivacyEngine()
    model, optimizer, train_loader = privacy_engine.make_private(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        noise_multiplier=noise_multiplier,
        max_grad_norm=max_grad_norm,
        poisson_sampling=False,
    )
    model.train()
    best_val_acc = 0.0
    patience_counter = 0
    best_model_state = None
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        scheduler.step()  # update learning rate after each epoch
        model.eval()
        correct, total = 0, 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_accuracy = 100 * correct / total
        avg_train_loss = running_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        if ((epoch + 1) % report_every == 0) or (epoch == num_epochs - 1):
            print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print(f"Early stopping at epoch {epoch+1}. Best val accuracy: {best_val_acc:.2f}%")
                if best_model_state is not None:
                    model.load_state_dict(best_model_state)
                break
        model.train()
    return model

# ---- DP k-fold training ----
def kfold_crossval_training_dp(train_data, device, k=3, batch_size=8, num_epochs=3,
                               noise_multiplier=0.1, max_grad_norm=1.0,
                               early_stopping_patience=3, report_every=1):
    targets = [label for _, label in train_data]
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    fold_models = []
    for fold, (train_indices, val_indices) in enumerate(skf.split(np.zeros(len(targets)), targets)):
        print(f"Fold {fold+1}/{k}")
        train_fold_subset = Subset(train_data, train_indices)
        val_fold_subset = Subset(train_data, val_indices)
        train_fold_loader = DataLoader(train_fold_subset, batch_size=batch_size, shuffle=True)
        val_fold_loader = DataLoader(val_fold_subset, batch_size=batch_size, shuffle=False)
        model = SimpleCNN_DP().to(device)
        class_counts = Counter([train_data[idx][1] for idx in train_indices])
        total_train = sum(class_counts.values())
        class_weights = [total_train / class_counts[i] if class_counts[i] > 0 else 0.0 for i in range(len(class_counts))]
        weights_tensor = torch.FloatTensor(class_weights).to(device)
        criterion = nn.CrossEntropyLoss(weight=weights_tensor)
        optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
        model = train_model_dp(
            model=model,
            train_loader=train_fold_loader,
            val_loader=val_fold_loader,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            noise_multiplier=noise_multiplier,
            max_grad_norm=max_grad_norm,
            num_epochs=num_epochs,
            early_stopping_patience=early_stopping_patience,
            report_every=report_every,
        )
        fold_models.append(model)
        # Clear memory after each fold
        gc.collect()
        torch.cuda.empty_cache()
    return fold_models

# ---- Ensemble model for soft voting ----
class EnsembleModel(nn.Module):
    def __init__(self, models):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList(models)
        for model in self.models:
            model.eval()
    def forward(self, x):
        outputs = [torch.softmax(m(x), dim=1) for m in self.models]
        mean_output = torch.mean(torch.stack(outputs), dim=0)
        return mean_output

# ---- Evaluate model (classification report + ROC curve) ----
def evaluate_model_extended(model, loader, device):
    model.eval()
    y_pred, y_true, scores = [], [], []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
            probs = torch.softmax(outputs, dim=1)
            scores.extend(probs[:,1].cpu().numpy())
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=["NORMAL", "PNEUMONIA"]))
    fpr, tpr, _ = roc_curve(y_true, scores)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
    plt.plot([0,1],[0,1],'--', color='gray')
    plt.title("ROC Curve")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend()
    plt.show()

# ---- Collect max softmax confidence for MIA ----
def collect_confidences(model, loader, device):
    model.eval()
    confidences, labels = [], []
    with torch.no_grad():
        for images, labs in loader:
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            max_confidence, _ = torch.max(probs, dim=1)
            confidences.extend(max_confidence.cpu().numpy())
            labels.extend(labs.cpu().numpy())
    return np.array(confidences), np.array(labels)

# ---- Main grid search with MIA attack ----
def grid_search_with_mia(train_data, train_loader, val_loader, test_loader, device, avg_grad_norm):
    noise_multipliers = [0.001, 0.01, 0.05]
    max_grad_norms = [avg_grad_norm * 0.5, avg_grad_norm, avg_grad_norm * 2]
    results = []
    for noise_mult in noise_multipliers:
        for max_grad in max_grad_norms:
            print(f"\nGrid search - noise_multiplier={noise_mult}, max_grad_norm={max_grad:.4f}")
            fold_models = kfold_crossval_training_dp(
                train_data=train_data,
                device=device,
                k=3,
                batch_size=8,
                num_epochs=3,
                noise_multiplier=noise_mult,
                max_grad_norm=max_grad,
                early_stopping_patience=3,
                report_every=1,
            )
            ensemble_model = EnsembleModel(fold_models).to(device)
            # Evaluate utility
            evaluate_model_extended(ensemble_model, test_loader, device)
            # Collect confidences for MIA attack
            train_confidences, _ = collect_confidences(ensemble_model, train_loader, device)
            test_confidences, _ = collect_confidences(ensemble_model, test_loader, device)
            mia_X = np.concatenate([train_confidences, test_confidences])[:, None]
            mia_y = np.concatenate([np.ones(len(train_confidences)), np.zeros(len(test_confidences))])
            attack_model = LogisticRegression(solver="lbfgs", max_iter=1000)
            attack_model.fit(mia_X, mia_y)
            mia_preds = attack_model.predict(mia_X)
            mia_probs = attack_model.predict_proba(mia_X)[:, 1]
            mia_acc = accuracy_score(mia_y, mia_preds)
            mia_auc = roc_auc_score(mia_y, mia_probs)
            print(f"MIA Attack Accuracy: {mia_acc:.3f}, ROC AUC: {mia_auc:.3f}")
            results.append({
                "noise_multiplier": noise_mult,
                "max_grad_norm": max_grad,
                "mia_attack_acc": mia_acc,
                "mia_attack_auc": mia_auc
            })
            # Clear memory explicitly after each grid point
            gc.collect()
            torch.cuda.empty_cache()
    # Save results to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv("dp_grid_search_results.csv", index=False)
    print("Grid search complete. Results saved to dp_grid_search_results.csv")
    # Plot results
    fig, ax1 = plt.subplots()
    ax1.set_xlabel('Noise Multiplier')
    ax1.set_ylabel('MIA Attack Accuracy', color='tab:red')
    ax1.plot(results_df['noise_multiplier'], results_df['mia_attack_acc'], 'o-', color='tab:red', label='MIA Attack Accuracy')
    ax1.tick_params(axis='y', labelcolor='tab:red')
    ax2 = ax1.twinx()
    ax2.set_ylabel('MIA Attack ROC AUC', color='tab:blue')
    ax2.plot(results_df['noise_multiplier'], results_df['mia_attack_auc'], 's--', color='tab:blue', label='MIA Attack ROC AUC')
    ax2.tick_params(axis='y', labelcolor='tab:blue')
    plt.title("MIA Attack Metrics vs Noise Multiplier")
    fig.tight_layout()
    plt.legend(loc="upper left")
    plt.show()

# Example usage:
sample_model = SimpleCNN_DP().to(device)
grad_analysis = analyze_gradient_magnitudes(sample_model, train_loader, device, num_batches=5)
avg_grad_norm = grad_analysis['avg_norm']
grid_search_with_mia(train_data, train_loader, val_loader, test_loader, device, avg_grad_norm)


### Differentially Private Training with ResNet18

- Loads and preprocesses chest X-ray images for binary classification.
- Uses pretrained ResNet18 with frozen layers; replaces and trains final fully connected layer.
- Applies grayscale-to-RGB conversion to match ResNet input requirements.
- Integrates `Opacus` PrivacyEngine to enforce differential privacy during training:
  - Adds calibrated noise to gradients.
  - Clips gradients to bound sensitivity.
  - Tracks privacy budget (ε, δ) across epochs.
- Trains model using cross-entropy loss and Adam optimizer.
- Evaluates model performance using classification report and ROC curve on validation set.

Below code basically an earlier/training-only script that leans on the tiny val/ folder and lacks MIA + final test reporting. You don’t need it for the dissertation.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from opacus import PrivacyEngine
import numpy as np
from sklearn.metrics import classification_report, roc_curve, auc
import matplotlib.pyplot as plt
from torchvision.models import ResNet18_Weights

# Dataset paths
base_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray"
train_dir = f"{base_dir}/train"
val_dir = f"{base_dir}/val"
test_dir = f"{base_dir}/test"

# Transforms (224x224, grayscale converted to 3-channel for ResNet)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # ResNet expects 3 channels
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

batch_size = 32  # adjust if needed

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained ResNet18 with updated API, freeze all layers first
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
for param in model.parameters():
    param.requires_grad = False

# Replace final fully connected layer (unfreeze it)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 classes: NORMAL, PNEUMONIA
for param in model.fc.parameters():
    param.requires_grad = True

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

# Initialize PrivacyEngine correctly (no args in constructor)
privacy_engine = PrivacyEngine()

model, optimizer, train_loader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    noise_multiplier=1.0,   # tune noise for privacy-utility tradeoff
    max_grad_norm=1.0,
    epochs=10,              # total epochs for privacy accounting
)

def evaluate(model, loader):
    model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_probs.extend(probs[:,1].cpu().numpy())
    print(classification_report(y_true, y_pred, target_names=train_dataset.classes))
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
    plt.plot([0,1], [0,1], '--', color='gray')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")
    
    epsilon = privacy_engine.get_epsilon(delta=1e-5)
    print(f"Privacy budget spent: ε = {epsilon:.2f}, δ=1e-5")
    
    print("Validation results:")
    evaluate(model, val_loader)



In [ ]:
# --------- Final Test Evaluation ---------
print("Final test results:")
evaluate(model, test_loader)

# --------- Collect confidence scores for MIA ---------
def collect_confidences(model, loader):
    model.eval()
    confidences = []
    labels = []
    with torch.no_grad():
        for imgs, labs in loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            probs = torch.softmax(outputs, dim=1)
            max_confidence, _ = torch.max(probs, dim=1)
            confidences.extend(max_confidence.cpu().numpy())
            labels.extend(labs.cpu().numpy())
    return np.array(confidences), np.array(labels)

# --------- MIA attack simulation ---------
print("Running Membership Inference Attack (MIA) simulation...")

train_confidences, _ = collect_confidences(model, train_loader)
test_confidences, _ = collect_confidences(model, test_loader)

# Labels: 1 for members (train), 0 for non-members (test)
mia_X = np.concatenate([train_confidences, test_confidences])[:, None]
mia_y = np.concatenate([np.ones(len(train_confidences)), np.zeros(len(test_confidences))])

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

attack_model = LogisticRegression(solver='lbfgs', max_iter=1000)
attack_model.fit(mia_X, mia_y)
mia_preds = attack_model.predict(mia_X)
mia_probs = attack_model.predict_proba(mia_X)[:, 1]

mia_acc = accuracy_score(mia_y, mia_preds)
mia_auc = roc_auc_score(mia_y, mia_probs)

print(f"MIA Attack Accuracy: {mia_acc:.3f}")
print(f"MIA Attack ROC AUC: {mia_auc:.3f}")


### Differentially Private Training and Membership Inference Attack (MIA)

#### Data Preparation
- Loads chest X-ray images from Kaggle dataset.
- Applies preprocessing: grayscale to RGB conversion, resizing to 224×224, and tensor conversion.
- Splits training data into 80% training and 20% validation.
- Creates DataLoaders for training, validation, and test sets.

#### Model Setup
- Loads pretrained ResNet18 and freezes all layers except the final fully connected layer.
- Replaces the final layer to output logits for two classes: NORMAL and PNEUMONIA.
- Moves model to GPU if available.

#### Differential Privacy Integration
- Uses `Opacus` PrivacyEngine to apply differentially private stochastic gradient descent (DP-SGD).
- Adds noise to gradients and clips them to limit sensitivity.
- Tracks privacy budget (ε, δ) across training epochs.

#### Training Loop
- Trains the model for 10 epochs using DP-SGD.
- Computes and prints average training loss per epoch.
- Reports privacy budget spent (ε) after each epoch.
- Evaluates model on validation set using classification metrics and ROC curve.

#### Final Evaluation
- Evaluates model performance on the held-out test set using classification report and ROC AUC.

#### Membership Inference Attack (MIA)
- Collects maximum softmax confidence scores from training and test sets.
- Constructs an attack dataset: confidence scores as features, membership labels (1 for train, 0 for test).
- Trains a logistic regression model to predict membership status.
- Evaluates attack success using accuracy and ROC AUC.
- High attack performance may indicate privacy leakage from the trained model.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from opacus import PrivacyEngine
import numpy as np
from sklearn.metrics import classification_report, roc_curve, auc, accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from torchvision.models import ResNet18_Weights

# Dataset paths
base_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray"
train_dir = f"{base_dir}/train"
test_dir = f"{base_dir}/test"

# Transforms (224x224, grayscale converted to 3-channel for ResNet)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # ResNet expects 3 channels
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load full training dataset (will split into train + val)
full_train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

# Split full training set into 80% train, 20% validation
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

print(f"Train samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}, Test samples: {len(test_dataset)}")
print(f"Classes: {full_train_dataset.classes}")

batch_size = 32  # adjust based on your GPU

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained ResNet18 and freeze all layers
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
for param in model.parameters():
    param.requires_grad = False

# Replace final FC layer and unfreeze it
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
for param in model.fc.parameters():
    param.requires_grad = True

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

# Setup Privacy Engine
privacy_engine = PrivacyEngine()
model, optimizer, train_loader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    noise_multiplier=1.0,   # Tune for your privacy-utility tradeoff
    max_grad_norm=1.0,
    epochs=10,              # total epochs for privacy accounting
)

def evaluate(model, loader):
    model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_probs.extend(probs[:,1].cpu().numpy())
    print(classification_report(y_true, y_pred, target_names=full_train_dataset.classes))
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
    plt.plot([0,1], [0,1], '--', color='gray')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()

# --------- Collect confidence scores for MIA ---------
def collect_confidences(model, loader):
    model.eval()
    confidences = []
    labels = []
    with torch.no_grad():
        for imgs, labs in loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            probs = torch.softmax(outputs, dim=1)
            max_confidence, _ = torch.max(probs, dim=1)
            confidences.extend(max_confidence.cpu().numpy())
            labels.extend(labs.cpu().numpy())
    return np.array(confidences), np.array(labels)

# Training loop with DP
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

    epsilon = privacy_engine.get_epsilon(delta=1e-5)
    print(f"Privacy budget spent: ε = {epsilon:.2f}, δ=1e-5")

    print("Validation results:")
    evaluate(model, val_loader)

print("Final test results:")
evaluate(model, test_loader)

# --------- MIA attack simulation ---------
print("Running Membership Inference Attack (MIA) simulation...")

train_confidences, _ = collect_confidences(model, train_loader)
test_confidences, _ = collect_confidences(model, test_loader)

# Labels: 1 for members (train), 0 for non-members (test)
mia_X = np.concatenate([train_confidences, test_confidences])[:, None]
mia_y = np.concatenate([np.ones(len(train_confidences)), np.zeros(len(test_confidences))])

from sklearn.linear_model import LogisticRegression

attack_model = LogisticRegression(solver='lbfgs', max_iter=1000)
attack_model.fit(mia_X, mia_y)
mia_preds = attack_model.predict(mia_X)
mia_probs = attack_model.predict_proba(mia_X)[:, 1]

mia_acc = accuracy_score(mia_y, mia_preds)
mia_auc = roc_auc_score(mia_y, mia_probs)

print(f"MIA Attack Accuracy: {mia_acc:.3f}")
print(f"MIA Attack ROC AUC: {mia_auc:.3f}")


Code In box below runs the grid search - takes 4 hours

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from opacus import PrivacyEngine
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import gc

# Dataset paths and transforms
base_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray"
train_dir = f"{base_dir}/train"
test_dir = f"{base_dir}/test"

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load datasets and split train/val
full_train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def create_data_loaders(batch_size):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    return train_loader, val_loader, test_loader

def build_model():
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    for param in model.parameters():
        param.requires_grad = False
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 2)
    for param in model.fc.parameters():
        param.requires_grad = True
    return model.to(device)

def train_and_evaluate(noise_multiplier, max_grad_norm, learning_rate, epochs=10):
    print(f"\nStarting training with noise_multiplier={noise_multiplier}, max_grad_norm={max_grad_norm}, learning_rate={learning_rate}")
    
    train_loader, val_loader, test_loader = create_data_loaders(batch_size)
    
    model = build_model()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

    privacy_engine = PrivacyEngine()
    model, optimizer, train_loader = privacy_engine.make_private(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        noise_multiplier=noise_multiplier,
        max_grad_norm=max_grad_norm,
        epochs=epochs,
    )

    best_val_acc = 0
    best_model_state = None

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                preds = outputs.argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total

        epsilon = privacy_engine.get_epsilon(delta=1e-5)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f} - Val Acc: {val_acc:.4f} - ε: {epsilon:.2f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict()

    # Load best model for test evaluation
    if best_model_state:
        model.load_state_dict(best_model_state)

    # Test evaluation
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    test_acc = accuracy_score(y_true, y_pred)
    print(f"Test Accuracy: {test_acc:.4f}")

    # Collect confidence scores for MIA
    train_confidences = collect_confidences(model, train_loader)
    test_confidences = collect_confidences(model, test_loader)

    mia_X = np.concatenate([train_confidences, test_confidences])[:, None]
    mia_y = np.concatenate([np.ones(len(train_confidences)), np.zeros(len(test_confidences))])
    attack_model = LogisticRegression(solver='lbfgs', max_iter=1000)
    attack_model.fit(mia_X, mia_y)
    mia_preds = attack_model.predict(mia_X)
    mia_probs = attack_model.predict_proba(mia_X)[:, 1]
    mia_acc = accuracy_score(mia_y, mia_preds)
    mia_auc = roc_auc_score(mia_y, mia_probs)
    print(f"MIA Attack Accuracy: {mia_acc:.3f}, ROC AUC: {mia_auc:.3f}")

    # Clear memory to avoid crashes
    del model, optimizer, privacy_engine, train_loader, val_loader, test_loader
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return val_acc, epsilon, mia_acc, mia_auc, noise_multiplier, max_grad_norm, learning_rate

def collect_confidences(model, loader):
    model.eval()
    confidences = []
    with torch.no_grad():
        for imgs, _ in loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            probs = torch.softmax(outputs, dim=1)
            max_confidence, _ = torch.max(probs, dim=1)
            confidences.extend(max_confidence.cpu().numpy())
    return np.array(confidences)

# Grid search parameter options
noise_multipliers = [0.5, 1.0, 1.5]
max_grad_norms = [0.5, 1.0, 2.0]
learning_rates = [1e-3, 5e-4]

results = []

for nm in noise_multipliers:
    for mg in max_grad_norms:
        for lr in learning_rates:
            val_acc, epsilon, mia_acc, mia_auc, nm_, mg_, lr_ = train_and_evaluate(nm, mg, lr)
            results.append({
                "val_accuracy": val_acc,
                "epsilon": epsilon,
                "mia_attack_accuracy": mia_acc,
                "mia_attack_auc": mia_auc,
                "noise_multiplier": nm_,
                "max_grad_norm": mg_,
                "learning_rate": lr_,
            })

# Save results
import pandas as pd
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by=["val_accuracy", "epsilon"], ascending=[False, True])
df_results.to_csv("dp_grid_search_results.csv", index=False)
print("Grid search complete. Results saved to dp_grid_search_results.csv")
print(df_results)


Section Below breaks down the cell above so I don't need to re run the grid search again

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from opacus import PrivacyEngine
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import gc
import pandas as pd


In [ ]:
# Dataset paths and transforms
base_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray"
train_dir = f"{base_dir}/train"
test_dir = f"{base_dir}/test"

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load datasets and split train/val
full_train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def create_data_loaders(batch_size):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    return train_loader, val_loader, test_loader


In [ ]:
def build_model():
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    for param in model.parameters():
        param.requires_grad = False
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 2)
    for param in model.fc.parameters():
        param.requires_grad = True
    return model.to(device)

def collect_confidences(model, loader):
    model.eval()
    confidences = []
    with torch.no_grad():
        for imgs, _ in loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            probs = torch.softmax(outputs, dim=1)
            max_confidence, _ = torch.max(probs, dim=1)
            confidences.extend(max_confidence.cpu().numpy())
    return np.array(confidences)


In [ ]:
def train_and_evaluate(noise_multiplier, max_grad_norm, learning_rate, epochs=15):
    print(f"\nStarting training with noise_multiplier={noise_multiplier}, max_grad_norm={max_grad_norm}, learning_rate={learning_rate}")
    
    train_loader, val_loader, test_loader = create_data_loaders(batch_size)
    
    model = build_model()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

    # Use secure_mode=False so it works in Kaggle (no need to specify, as it's default)
    privacy_engine = PrivacyEngine(secure_mode=False)
    model, optimizer, train_loader = privacy_engine.make_private(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        noise_multiplier=noise_multiplier,
        max_grad_norm=max_grad_norm,
        epochs=epochs,
    )

    best_val_acc = 0
    best_model_state = None

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                preds = outputs.argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total

        epsilon = privacy_engine.get_epsilon(delta=1e-5)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f} - Val Acc: {val_acc:.4f} - ε: {epsilon:.2f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict()

    # Load best model for test evaluation
    if best_model_state:
        model.load_state_dict(best_model_state)
        torch.save(model.state_dict(), "best_dp_medical_model.pth")
        print("Final model saved as best_dp_medical_model.pth")

    # Test evaluation
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    test_acc = accuracy_score(y_true, y_pred)
    print(f"Test Accuracy: {test_acc:.4f}")

    # Collect confidence scores for MIA
    train_confidences = collect_confidences(model, train_loader)
    test_confidences = collect_confidences(model, test_loader)

    mia_X = np.concatenate([train_confidences, test_confidences])[:, None]
    mia_y = np.concatenate([np.ones(len(train_confidences)), np.zeros(len(test_confidences))])
    attack_model = LogisticRegression(solver='lbfgs', max_iter=1000)
    attack_model.fit(mia_X, mia_y)
    mia_preds = attack_model.predict(mia_X)
    mia_probs = attack_model.predict_proba(mia_X)[:, 1]
    mia_acc = accuracy_score(mia_y, mia_preds)
    mia_auc = roc_auc_score(mia_y, mia_probs)
    print(f"MIA Attack Accuracy: {mia_acc:.3f}, ROC AUC: {mia_auc:.3f}")

    # Clear memory to avoid crashes
    del model, optimizer, privacy_engine, train_loader, val_loader, test_loader
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return best_val_acc, epsilon, mia_acc, mia_auc


In [ ]:
# Train ONLY with the best settings!
final_val_acc, final_epsilon, final_mia_acc, final_mia_auc = train_and_evaluate(
    noise_multiplier=1.5,
    max_grad_norm=1.0,
    learning_rate=0.001,
    epochs=15  # or 10/20 depending on what you want
)
print(f"\nFinal Model - Val Acc: {final_val_acc:.4f}, Epsilon: {final_epsilon:.3f}, "
      f"MIA Acc: {final_mia_acc:.3f}, MIA AUC: {final_mia_auc:.3f}")


Below code is working on the imbalance in the dataset to get better results- That code is running a balanced Membership Inference Attack (MIA) test using the confidence scores your model outputs

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_fscore_support

# assumes you already have:
# train_confidences, test_confidences = collect_confidences(model, loader) from your script

def evaluate_mia_balanced(train_confidences, test_confidences, seed=42):
    rng = np.random.default_rng(seed)
    n = min(len(train_confidences), len(test_confidences))

    idx_tr = rng.choice(len(train_confidences), n, replace=False)
    idx_te = rng.choice(len(test_confidences), n, replace=False)

    X = np.concatenate([train_confidences[idx_tr], test_confidences[idx_te]])[:, None]
    y = np.concatenate([np.ones(n), np.zeros(n)])

    attack = LogisticRegression(solver='lbfgs', max_iter=1000)
    attack.fit(X, y)
    preds = attack.predict(X)
    probs = attack.predict_proba(X)[:, 1]

    acc = accuracy_score(y, preds)
    auc = roc_auc_score(y, probs)
    prec, rec, f1, _ = precision_recall_fscore_support(y, preds, average='binary')

    print(f"[Balanced MIA] Acc: {acc:.3f} | AUC: {auc:.3f} | Precision: {prec:.3f} | Recall: {rec:.3f} | F1: {f1:.3f}")

# call it:
evaluate_mia_balanced(train_confidences, test_confidences)


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

def mia_balanced_once(train_confidences, test_confidences, seed):
    rng = np.random.default_rng(seed)
    n = min(len(train_confidences), len(test_confidences))
    X = np.concatenate([
        rng.choice(train_confidences, n, replace=False),
        rng.choice(test_confidences,  n, replace=False)
    ])[:, None]
    y = np.concatenate([np.ones(n), np.zeros(n)])
    clf = LogisticRegression(solver='lbfgs', max_iter=1000)
    clf.fit(X, y)
    probs = clf.predict_proba(X)[:,1]
    preds = (probs >= 0.5).astype(int)
    return accuracy_score(y, preds), roc_auc_score(y, probs)

seeds = range(20)
accs, aucs = zip(*(mia_balanced_once(train_confidences, test_confidences, s) for s in seeds))
print(f"Balanced MIA (20 runs) — Acc: {np.mean(accs):.3f}±{np.std(accs):.3f}, AUC: {np.mean(aucs):.3f}±{np.std(aucs):.3f}")


A balanced membership inference attack (MIA) was conducted over 20 independent runs to remove dataset size bias.
The attack achieved a mean accuracy of 0.540 ± 0.007 and a mean AUC of 0.517 ± 0.009.
As both values are close to the random-guessing baseline (0.5), this indicates that the model’s output confidences contain minimal distinguishable information about training set membership, suggesting low privacy leakage under this attack scenario.

In [ ]:
# ================================
# Shadow-Model MIA for Chest X-rays (ResNet18 + Opacus for target)
# ================================
import os
import gc
import math
import numpy as np
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets, transforms, models
from torchvision.models import ResNet18_Weights

from opacus import PrivacyEngine

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# -------------------------
# Config
# -------------------------
BASE_DIR = "/kaggle/input/chest-xray-pneumonia/chest_xray"
TRAIN_DIR = f"{BASE_DIR}/train"
TEST_DIR  = f"{BASE_DIR}/test"

BATCH_SIZE     = 32
EPOCHS_TARGET  = 10
EPOCHS_SHADOW  = 8
LR             = 1e-3
DP_NOISE       = 1.0      # DP noise (target only)
MAX_GRAD_NORM  = 1.0
VAL_RATIO_TGT  = 0.2      # % of target split for validation
SEED           = 42
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
np.random.seed(SEED)

# -------------------------
# Transforms
# -------------------------
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # ResNet expects 3 channels
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# -------------------------
# Data
# -------------------------
full_train = datasets.ImageFolder(TRAIN_DIR, transform=transform)
test_ds    = datasets.ImageFolder(TEST_DIR,  transform=transform)
classes    = full_train.classes
print(f"Classes: {classes}")

# Split full_train -> target_pool (60%), shadow_pool (40%)
tgt_size   = int(0.6 * len(full_train))
shd_size   = len(full_train) - tgt_size
target_pool, shadow_pool = random_split(full_train, [tgt_size, shd_size], generator=torch.Generator().manual_seed(SEED))

# Further split target_pool -> target_train (80%), target_val (20%)
tgt_train_size = int((1.0 - VAL_RATIO_TGT) * len(target_pool))
tgt_val_size   = len(target_pool) - tgt_train_size
target_train, target_val = random_split(target_pool, [tgt_train_size, tgt_val_size], generator=torch.Generator().manual_seed(SEED))

print(f"Target train: {len(target_train)} | Target val: {len(target_val)} | Shadow pool: {len(shadow_pool)} | Test: {len(test_ds)}")

# Shadow split -> shadow_train (80%), shadow_val (20%)
shd_train_size = int(0.8 * len(shadow_pool))
shd_val_size   = len(shadow_pool) - shd_train_size
shadow_train, shadow_val = random_split(shadow_pool, [shd_train_size, shd_val_size], generator=torch.Generator().manual_seed(SEED))
print(f"Shadow train: {len(shadow_train)} | Shadow val: {len(shadow_val)}")

# DataLoaders
def make_loader(ds, bs=BATCH_SIZE, shuffle=False):
    return DataLoader(ds, batch_size=bs, shuffle=shuffle, num_workers=2, pin_memory=True)

tgt_train_loader = make_loader(target_train, shuffle=True)
tgt_val_loader   = make_loader(target_val)
test_loader      = make_loader(test_ds)

shd_train_loader = make_loader(shadow_train, shuffle=True)
shd_val_loader   = make_loader(shadow_val)

# -------------------------
# Models
# -------------------------
def build_resnet18_head2():
    m = models.resnet18(weights=ResNet18_Weights.DEFAULT)
    for p in m.parameters(): p.requires_grad = False
    in_f = m.fc.in_features
    m.fc = nn.Linear(in_f, 2)
    for p in m.fc.parameters(): p.requires_grad = True
    return m.to(DEVICE)

# -------------------------
# Train loops
# -------------------------
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    run_loss = 0.0
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        run_loss += loss.item()
    return run_loss / max(1, len(loader))

@torch.no_grad()
def eval_acc(model, loader):
    model.eval()
    correct = 0
    total = 0
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        preds = out.argmax(1)
        correct += (preds == y).sum().item()
        total += y.size(0)
    return correct / max(1, total)

# -------------------------
# TARGET: DP training
# -------------------------
target_model = build_resnet18_head2()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(target_model.fc.parameters(), lr=LR)

privacy_engine = PrivacyEngine()
target_model, optimizer, tgt_train_loader = privacy_engine.make_private(
    module=target_model,
    optimizer=optimizer,
    data_loader=tgt_train_loader,
    noise_multiplier=DP_NOISE,
    max_grad_norm=MAX_GRAD_NORM,
    epochs=EPOCHS_TARGET,   # for accounting
)

print("\n--- Training DP Target ---")
best_val = 0.0
best_state = None
for ep in range(1, EPOCHS_TARGET+1):
    loss = train_epoch(target_model, tgt_train_loader, optimizer, criterion)
    val_acc = eval_acc(target_model, tgt_val_loader)
    eps = privacy_engine.get_epsilon(delta=1e-5)
    print(f"Epoch {ep}/{EPOCHS_TARGET} - Loss: {loss:.4f} - Val Acc: {val_acc:.4f} - ε: {eps:.2f}")
    if val_acc > best_val:
        best_val = val_acc
        best_state = target_model.state_dict()

if best_state:
    target_model.load_state_dict(best_state)
torch.save(target_model.state_dict(), "dp_target_model.pth")
print(f"Saved target model. Best Val Acc: {best_val:.4f}")

# Evaluate on test set (utility)
test_acc = eval_acc(target_model, test_loader)
print(f"Target Test Accuracy: {test_acc:.4f}")

# -------------------------
# SHADOW: Non-DP training (attacker’s proxy of your pipeline)
# -------------------------
shadow_model = build_resnet18_head2()
shd_criterion = nn.CrossEntropyLoss()
shd_optimizer = optim.Adam(shadow_model.fc.parameters(), lr=LR)

print("\n--- Training Shadow (non-DP) ---")
best_shd = 0.0
best_shd_state = None
for ep in range(1, EPOCHS_SHADOW+1):
    loss = train_epoch(shadow_model, shd_train_loader, shd_optimizer, shd_criterion)
    val_acc = eval_acc(shadow_model, shd_val_loader)
    print(f"[Shadow] Epoch {ep}/{EPOCHS_SHADOW} - Loss: {loss:.4f} - Val Acc: {val_acc:.4f}")
    if val_acc > best_shd:
        best_shd = val_acc
        best_shd_state = shadow_model.state_dict()

if best_shd_state:
    shadow_model.load_state_dict(best_shd_state)
torch.save(shadow_model.state_dict(), "shadow_model.pth")
print(f"Saved shadow model. Best Shadow Val Acc: {best_shd:.4f}")

# -------------------------
# Attack feature extraction
# -------------------------
@torch.no_grad()
def collect_features(model, loader):
    """
    Returns:
      feats: [N, 2 + 3] => [p0, p1, loss, entropy, max_prob]
      labs : labels (not used by attacker except for ref)
    """
    model.eval()
    feats = []
    labs  = []
    ce = nn.CrossEntropyLoss(reduction="none")
    for x, y in loader:
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        logits = model(x)
        probs  = torch.softmax(logits, dim=1)
        # per-sample loss
        losses = ce(logits, y)
        # entropy
        entropy = -(probs * torch.log(probs.clamp_min(1e-12))).sum(dim=1)
        # max prob
        maxprob, _ = probs.max(dim=1)
        # features: full probs + loss + entropy + maxprob
        feat_batch = torch.cat([probs, losses.unsqueeze(1), entropy.unsqueeze(1), maxprob.unsqueeze(1)], dim=1)
        feats.append(feat_batch.cpu().numpy())
        labs.extend(y.cpu().numpy())
    feats = np.concatenate(feats, axis=0) if len(feats) else np.zeros((0, 5))
    labs  = np.array(labs)
    return feats, labs

# Train attacker on shadow model signals:
# member = shadow_train; non-member = shadow_val
shadow_member_feats, _     = collect_features(shadow_model, shd_train_loader)
shadow_nonmember_feats, _  = collect_features(shadow_model, shd_val_loader)

X_attack_train = np.vstack([shadow_member_feats, shadow_nonmember_feats])
y_attack_train = np.concatenate([np.ones(len(shadow_member_feats)), np.zeros(len(shadow_nonmember_feats))])

# -------------------------
# Fit attack model
# -------------------------
attack_clf = LogisticRegression(max_iter=2000, solver="lbfgs")
attack_clf.fit(X_attack_train, y_attack_train)

# -------------------------
# Evaluate attacker on TARGET model:
# member = target_train; non-member = test
# -------------------------
target_member_feats, _    = collect_features(target_model, tgt_train_loader)
target_nonmember_feats, _ = collect_features(target_model, test_loader)

X_attack_test = np.vstack([target_member_feats, target_nonmember_feats])
y_attack_test = np.concatenate([np.ones(len(target_member_feats)), np.zeros(len(target_nonmember_feats))])

attack_preds = attack_clf.predict(X_attack_test)
attack_probs = attack_clf.predict_proba(X_attack_test)[:, 1]

mia_acc = accuracy_score(y_attack_test, attack_preds)
mia_auc = roc_auc_score(y_attack_test, attack_probs)

print("\n=== Shadow-Model MIA Results (on TARGET) ===")
print(f"Attack Accuracy: {mia_acc:.3f}")
print(f"Attack ROC AUC: {mia_auc:.3f}")

# Optional: quick report of target utility
# (You can add a full classification_report using predictions if needed)
print(f"\nTarget test accuracy (utility): {test_acc:.3f} | Shadow val acc: {best_shd:.3f}")

# -------------------------
# Clean up CUDA memory
# -------------------------
del target_model, shadow_model, optimizer, shd_optimizer, privacy_engine
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()


In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from opacus import PrivacyEngine
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
import gc, math

# ---------------------- Config ----------------------
base_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray"
train_dir, test_dir = f"{base_dir}/train", f"{base_dir}/test"
batch_size = 32
epochs = 12                 # will early-stop
noise_multiplier = 1.7      # ↑ for more privacy, try 2.0 as well
max_grad_norm = 0.5
lr = 8e-4
weight_decay = 5e-4
label_smoothing = 0.1
mixup_alpha = 0.4           # 0 disables mixup
patience = 3                # early stop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------------------- Data -----------------------
tfm = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

full_train = datasets.ImageFolder(train_dir, transform=tfm)
test_set   = datasets.ImageFolder(test_dir, transform=tfm)
train_size = int(0.8 * len(full_train))
val_size   = len(full_train) - train_size
train_set, val_set = random_split(full_train, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_set,   batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_set,  batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

classes = full_train.classes
print(f"Train: {len(train_set)} | Val: {len(val_set)} | Test: {len(test_set)} | Classes: {classes}")

# ---------------------- Model ----------------------
def build_model(dropout_p=0.5):
    m = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    for p in m.parameters():
        p.requires_grad = False
    in_f = m.fc.in_features
    m.fc = nn.Sequential(
        nn.Dropout(dropout_p),
        nn.Linear(in_f, 2),
    )
    return m.to(device)

# ---------------------- MixUp ----------------------
def mixup_batch(x, y, alpha):
    if alpha <= 0:
        return x, y.float(), torch.ones(len(y), device=x.device)
    lam = np.random.beta(alpha, alpha)
    idx = torch.randperm(x.size(0), device=x.device)
    x_mixed = lam * x + (1 - lam) * x[idx]
    y_onehot = torch.nn.functional.one_hot(y, num_classes=2).float()
    y_mixed = lam * y_onehot + (1 - lam) * y_onehot[idx]
    return x_mixed, y_mixed, torch.full((len(y),), lam, device=x.device)

# Soft CE to handle mixup + label smoothing in one go
class SoftTargetCrossEntropy(nn.Module):
    def __init__(self, label_smoothing=0.0):
        super().__init__()
        self.eps = label_smoothing
    def forward(self, logits, targets):  # targets: one-hot or soft labels (B, C)
        # Smooth the targets
        if targets.dim() == 1:
            targets = torch.nn.functional.one_hot(targets, num_classes=logits.size(1)).float()
        targets = (1 - self.eps) * targets + self.eps / logits.size(1)
        log_probs = torch.nn.functional.log_softmax(logits, dim=1)
        return -(targets * log_probs).sum(dim=1).mean()

# ---------------------- Train (DP) ----------------------
def train_dp_model():
    model = build_model(dropout_p=0.5)
    criterion = SoftTargetCrossEntropy(label_smoothing=label_smoothing)
    optimizer = optim.Adam(model.fc.parameters(), lr=lr, weight_decay=weight_decay)

    privacy_engine = PrivacyEngine()
    model, optimizer, priv_train_loader = privacy_engine.make_private(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        noise_multiplier=noise_multiplier,
        max_grad_norm=max_grad_norm,
        epochs=epochs,
    )

    best_val = -1.0
    best_state = None
    patience_left = patience

    for ep in range(1, epochs+1):
        model.train()
        run_loss, n = 0.0, 0
        for x, y in priv_train_loader:
            x, y = x.to(device), y.to(device)

            # mixup
            x_mix, y_soft, _ = mixup_batch(x, y, mixup_alpha)

            optimizer.zero_grad()
            logits = model(x_mix)
            loss = criterion(logits, y_soft)
            loss.backward()
            optimizer.step()

            b = x.size(0)
            run_loss += loss.item() * b
            n += b

        # validation
        model.eval()
        y_true, y_pred, y_prob = [], [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                logits = model(x)
                probs = torch.softmax(logits, dim=1)
                preds = probs.argmax(dim=1)
                y_true.extend(y.cpu().numpy())
                y_pred.extend(preds.cpu().numpy())
                y_prob.extend(probs[:,1].cpu().numpy())
        val_acc = accuracy_score(y_true, y_pred)
        epsilon = privacy_engine.get_epsilon(delta=1e-5)
        print(f"Epoch {ep}/{epochs} - Loss: {run_loss/max(1,n):.4f} - Val Acc: {val_acc:.4f} - ε: {epsilon:.2f}")

        if val_acc > best_val + 1e-4:
            best_val = val_acc
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            patience_left = patience
        else:
            patience_left -= 1
            if patience_left <= 0:
                print(f"Early stopping at epoch {ep}. Best Val Acc: {best_val:.4f}")
                break

    # load best
    if best_state is not None:
        model.load_state_dict({k: v.to(device) for k, v in best_state.items()})
    torch.save(model.state_dict(), "dp_resnet18_privacy_aware.pth")
    return model, epsilon

# ---------------------- Evaluate ----------------------
def evaluate(model, loader, title="Eval"):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            preds = logits.argmax(dim=1)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    acc = accuracy_score(y_true, y_pred)
    print(f"{title} Accuracy: {acc:.4f}")
    print(classification_report(y_true, y_pred, target_names=classes, digits=3))
    return acc

# ---------------------- MIA: Balanced confidence attack ----------------------
def collect_confidences(model, loader):
    model.eval()
    conf = []
    with torch.no_grad():
        for x, _ in loader:
            x = x.to(device)
            probs = torch.softmax(model(x), dim=1)
            maxp, _ = probs.max(dim=1)
            conf.extend(maxp.cpu().numpy())
    return np.array(conf)

def mia_balanced(train_conf, test_conf, runs=10, seed=42):
    rng = np.random.default_rng(seed)
    from sklearn.metrics import roc_auc_score
    accs, aucs = [], []
    for _ in range(runs):
        n = min(len(train_conf), len(test_conf))
        tr_idx = rng.choice(len(train_conf), n, replace=False)
        te_idx = rng.choice(len(test_conf), n, replace=False)
        X = np.concatenate([train_conf[tr_idx], test_conf[te_idx]])[:, None]
        y = np.concatenate([np.ones(n), np.zeros(n)])
        clf = LogisticRegression(solver="lbfgs", max_iter=1000)
        clf.fit(X, y)
        preds = clf.predict(X)
        probs = clf.predict_proba(X)[:,1]
        accs.append(accuracy_score(y, preds))
        aucs.append(roc_auc_score(y, probs))
    return np.mean(accs), np.std(accs), np.mean(aucs), np.std(aucs)

# ---------------------- Run ----------------------
model, eps = train_dp_model()
val_acc = evaluate(model, val_loader, title="Validation")
test_acc = evaluate(model, test_loader, title="Test")

tr_conf = collect_confidences(model, train_loader)
te_conf = collect_confidences(model, test_loader)
acc_m, acc_sd, auc_m, auc_sd = mia_balanced(tr_conf, te_conf, runs=20)
print(f"Balanced MIA (20 runs) — Acc: {acc_m:.3f}±{acc_sd:.3f}, AUC: {auc_m:.3f}±{auc_sd:.3f}")

# house-keeping
gc.collect()
if torch.cuda.is_available(): torch.cuda.empty_cache()


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def plot_confusion_matrix(model, loader, class_names):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues, values_format='d')
    plt.title("Confusion Matrix")
    plt.show()

# Example usage:
# Define class names manually (order must match label encoding)
class_names = ['NORMAL', 'PNEUMONIA']

# Validation confusion matrix
plot_confusion_matrix(model, val_loader, class_names)

# Test confusion matrix
plot_confusion_matrix(model, test_loader, class_names)
